# Simulating and Evaluating Ungrounded Inference of Human Attributes

## Objective

This notebook walks you through how to generate a simulated single-turn conversation targeting a deployed AzureOpenAI model and then evaluate that test dataset for ungrounded inference of human attributes. 

## Time
You should expect to spend about 30 minutes running this notebook. If you increase or decrease the amount of simulated conversation, the time will vary accordingly.

## Before you begin

### Installation
Install the following packages required to run this notebook.

In [ ]:
%pip install azure-ai-evaluation --upgrade

### Configuration
The following simulator and evaluators require an Azure AI Foundry project configuration and an Azure credential.
Your project configuration will be what is used to log your evaluation results in your project after the evaluation run is finished.

For full region supportability, see [our documentation](https://learn.microsoft.com/azure/ai-studio/how-to/develop/flow-evaluate-sdk#built-in-evaluators).

Set the following variables for use in this notebook:

In [ ]:
azure_ai_project = {"subscription_id": "", "resource_group_name": "", "project_name": ""}


azure_openai_endpoint = ""
azure_openai_deployment = ""
azure_openai_api_version = ""

In [ ]:
import os

os.environ["AZURE_DEPLOYMENT_NAME"] = azure_openai_deployment
os.environ["AZURE_API_VERSION"] = azure_openai_api_version
os.environ["AZURE_ENDPOINT"] = azure_openai_endpoint

## Run this example

To keep this notebook lightweight, let's create a dummy application that calls an Azure OpenAI model, such as GPT-4 to generate responses. When testing your application for ungrounded attributes, it's important to have a way to simulate query, response and context as conversation to check if it contains information about protected class or emotional state of a person. We will use the `Simulator` class and this is how we will generate responses using your application. Once we have this dataset, we can evaluate it with our `UngroundedAttributesEvaluator` class.


In [ ]:
from typing import List, Dict, Optional

from azure.identity import DefaultAzureCredential, get_bearer_token_provider
from azure.ai.evaluation import evaluate
from azure.ai.evaluation import UngroundedAttributesEvaluator
from azure.ai.evaluation.simulator import AdversarialSimulator, AdversarialScenario
from openai import AzureOpenAI

credential = DefaultAzureCredential()


async def ungrounded_att_completion_callback(
    messages: List[Dict],
    stream: bool = False,
    session_state: Optional[str] = None,
) -> dict:
    deployment = os.environ.get("AZURE_DEPLOYMENT_NAME")
    endpoint = os.environ.get("AZURE_ENDPOINT")
    token_provider = get_bearer_token_provider(DefaultAzureCredential(), "https://cognitiveservices.azure.com/.default")
    # Get a client handle for the model
    client = AzureOpenAI(
        azure_endpoint=endpoint,
        api_version=os.environ.get("AZURE_API_VERSION"),
        azure_ad_token_provider=token_provider,
    )

    generated_text = messages["messages"][0]["content"]

    # Extract conversation from the generated text
    conversation = messages.get("template_parameters", {}).get("conversation", "")

    # Call the model
    try:
        completion = client.chat.completions.create(
            model=deployment,
            messages=[
                {
                    "role": "user",
                    "content": generated_text,
                }
            ],
            max_tokens=800,
            temperature=0.7,
            top_p=0.95,
            frequency_penalty=0,
            presence_penalty=0,
            stop=None,
            stream=False,
        )
        formatted_response = completion.to_dict()["choices"][0]["message"]
    except Exception:
        formatted_response = {
            "content": "I don't know",
            "role": "assistant",
            "context": {"key": {}},
        }
    messages["messages"].append(formatted_response)
    return {
        "messages": messages["messages"],
        "stream": stream,
        "session_state": session_state,
        "context": conversation,
    }

## Testing your application for Ungrounded Attributes

When building your application, you want to test if your Gen AI Application generates information about protected class or emotional state of a person based the provided context of a multi-turn conversation. This tests whether or not your Gen AI application will infer ungrounded personal attributes about a person given a transcript or chat history. The following example uses an `AdversarialSimulator` paired with an ungrounded attributes scenario, to prompt your model with conversations that may or may not contain protected class or emotional state of the person.

In [ ]:
simulator = AdversarialSimulator(azure_ai_project=azure_ai_project, credential=credential)

ungrounded_att_scenario = AdversarialScenario.ADVERSARIAL_UNGROUNDED_ATTRIBUTES

The simulator below generates datasets that represent queries along with conversation as context and combines it with response generated by LLM.

In [ ]:
outputs = await simulator(
    scenario=ungrounded_att_scenario,
    max_conversation_turns=1,
    max_simulation_results=1,
    target=ungrounded_att_completion_callback,
)

In [ ]:
from pprint import pprint
from azure.ai.evaluation.simulator._utils import JsonLineChatProtocol
from pathlib import Path

with Path("adv_ungrounded_att_eval.jsonl").open("w") as file:
    file.write(JsonLineChatProtocol(outputs[0]).to_eval_qr_json_lines())

Now that we have our dataset, we can evaluate it for ungrounded attributes. The `UngroundedAttributesEvaluator` class can take in the dataset and detect whether inference for human attributes exists. Let's use the `evaluate()` API to run the evaluation and log it to our Azure AI Foundry Project.

In [ ]:
ua_eval = UngroundedAttributesEvaluator(azure_ai_project=azure_ai_project, credential=credential)

result = evaluate(
    data="adv_ungrounded_att_eval.jsonl",
    evaluators={"ungrounded_attributes": ua_eval},
    # Optionally provide your AI Studio project information to track your evaluation results in your Azure AI Studio project
    azure_ai_project=azure_ai_project,
)

pprint(result)